# **0. 데이터 불러오기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install transformers -i https://pypi.python.org/simple

In [ ]:
import pandas as pd
import numpy as np
# 경고무시
import warnings
warnings.filterwarnings('ignore')
# 정규표현식
import re

import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained ('gogamza/kobart-summarization')

In [ ]:
df_policy = pd.read_excel('/content/drive/MyDrive/Wello+project_신희택,윤다솜/Data/policy_metetable(AIB전달용).xlsx')
df_policy = df_policy.sort_values(by='정책ID').reset_index(drop=True)
# 결론적으로 분석에 큰 지장이 없을 것 같아 소관기관유형.1과 소관기관.1을 삭제하도록함.
df_policy = df_policy.drop(['소관기관유형.1', '소관기관.1'],axis=1)
df_policy.head()

In [ ]:
# 많이 언급되는 지원대상, ○, :, \n, | 등

def clean_support_target(var):
  var = var.lstrip()
  var = var.replace('지원대상','').replace('○','').replace(':','').replace('\n','').replace('O','').replace('|','').replace('  ',' ').replace(' 소득','소득').replace('\xa0','').replace('*','').replace('-','')
  return var
df_policy['지원대상']=df_policy['지원대상'].apply(clean_support_target)

# **1. KoBART 코드**

In [ ]:
# 텍스트 클리닝
def cleaning_and_modeling(text):
  if '(' in text:
    text = re.sub(r'\([^)]*\)','',text)
    if 'ㆍ' in text:
      text = re.sub(r'\ㆍ[^)]*\ㆍ','',text)
    else:
      pass
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  summary_ids = model.generate(torch.tensor([input_ids]))
  return tokenizer.decode(summary_ids.squeeze().tolist(),skip_special_tokens=True)

In [ ]:
# 기업 관련 정책 뽑기
for i in range(len(df_policy)):
    if (('기업' in df_policy.loc[i, '지원대상'])|('중소' in df_policy.loc[i, '지원대상'])|('중견' in df_policy.loc[i, '지원대상'])
    |('법인' in df_policy.loc[i, '지원대상'])|('사업체' in df_policy.loc[i, '지원대상'])|('사업장' in df_policy.loc[i, '지원대상'])):
        df_policy.loc[i, 'company_policy']='기업'

In [ ]:
df_policy['지원내용'].apply(cleaning_and_modeling)